<a href="https://colab.research.google.com/github/DiamondTusks/Cybersecurity/blob/main/CSProg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linux Lab

### **Find and initialise your new storage device**

Use the lsblk command to view the block devices and find your new storage device.

```
sudo lsblk
```

Let's assume your new storage device is /dev/sdb. 
Initialise the disk with fdisk.

```
sudo fdisk /dev/sdb
```

Inside the fdisk utility:

press `n` to create a new partition,

press `p` for primary,

then `1` for the first partition. 

Press `Enter` to accept the default values for the first sector.

If you plan to have more partitions, follow instructions on screen, eg. type `+10G` for last sector if you want to allocate 10GB to the partition. Otherwise just press `Enter`.

Press `w` to write the partition table and exit.


### **Mount your new storage device to /var/www/html/wordpress/**

Create the mount point.

```
sudo mkdir -p /var/www/html/wordpress/
```

Mount the partition to this directory.

```
sudo mount /dev/sdb1 /var/www/html/wordpress/
```




###**Deploy WordPress to the `/var/www/html/wordpress/` directory and install it**

We are using the LAMP stack software bundle.

LAMP stands for Linux, Apache, MySQL, and PHP. Together, they provide a powerful web server environment. 

Here are the general steps to install a full LAMP stack and deploy WordPress on CentOS Stream 9. 

MySQL has already been deployed. You can see it with `systemctl status mysqld`. Database name, username and password has been set to `wordpress`.

Please note that this is a general guide, and depending on your specific setup and needs, you might need to adjust these steps.

1. **Update your system**: Keep your system up-to-date.

    ```bash
    sudo dnf update -y
    ```

2. **Install Apache**: Apache is a web server software.

    ```bash
    sudo dnf install httpd -y
    ```
    
    After installation, you need to start and enable Apache to make sure it starts automatically at system boot.

    ```bash
    sudo systemctl start httpd
    sudo systemctl enable httpd
    ```

3. **Install PHP**: PHP is a popular scripting language that is especially suited to web development.

    ```bash
    sudo dnf install php php-mysqlnd -y
    ```

    After the installation, restart Apache to make sure it recognizes and uses PHP.

    ```bash
    sudo systemctl restart httpd
    ```

5. **Download and Install WordPress**: 

    First, navigate to the Apache document root.

    ```bash
    cd /var/www/html/
    ```
    Download `wget` is required:
    ```bash
    sudo dnf install wget
    ```

    Download WordPress.

    ```bash
    sudo wget http://wordpress.org/latest.tar.gz
    ```

    Extract the downloaded file.

    ```bash
    sudo tar xzvf latest.tar.gz
    ```
    
    Remove compressed file.
    ```bash
    sudo rm latest.tar.gz
    ```

    Now, you should have a WordPress directory in `/var/www/html/`.

6. **Configure WordPress**:

    Move to the WordPress directory.

    ```bash
    cd /var/www/html/wordpress/
    ```

    Copy the sample WordPress config file to create your own config file.

    ```bash
    sudo cp wp-config-sample.php wp-config.php
    ```

    Open the WordPress config file and replace the 'database_name_here', 'username_here', and 'password_here' placeholders with your database details.

    ```bash
    sudo vi wp-config.php
    ```

8. **Update httpd.conf**:
    The main Apache (httpd) configuration file is usually found at `/etc/httpd/conf/httpd.conf` on CentOS systems. You can open and edit it using a text editor such as `nano` or `vi`. Here is how you can do it with `vi`:

    ```bash
    sudo vi /etc/httpd/conf/httpd.conf
    ```

    **DocumentRoot**: This is the directory out of which you will serve your documents. By default, all requests are taken from this directory, but symbolic links and aliases may be used to point to other locations.

    ```conf
    DocumentRoot "/var/www/html/wordpress"
    ```
    Press `i` to edit/insert
    Press `esc` then to save and exit type `:wq` and press ENTER

    Remember to always back up your configuration files before making changes.

    After making changes to the configuration file, you need to restart Apache to apply the changes:

    ```bash
    sudo systemctl restart httpd
    ```


9. **Finalizing Installation via Web Interface**:

    Finally, open your web browser and navigate to your server's domain name or public IP address followed by /wordpress: `http://domain_or_IP`.

    Follow the WordPress web interface to finalize the installation. This will include setting up a site name, username, password, and email.


###**Configure permissions so that WordPress owns its own files**

When you install WordPress, the files and folders are typically owned by the web server user. This can be a security risk, as it allows the web server user to modify or delete WordPress files. 

To improve security, it is best to configure WordPress to own its own files.

To do this, you will need to change the ownership of the files and folders to the user that owns the WordPress process. 

This can be done using an FTP client or SSH. Once you have changed the ownership, you will need to change the permissions on the files and folders. 

The permissions should be set to 644 for files and 755 for folders.

1.  **Change the ownership of the files and directories**:
    ```bash
    sudo chown -R apache:apache /var/www/html/wordpress/
    ```
  
2.  **Limit permissions for wp-config.php**:
    ```bash
    sudo chmod 400 /var/www/html/wordpress/wp-config.php
    ```
3.  **File and directory permissions**:

    All other file permissions should be left at their default values.

    By default, files should be 644, and directories should be 755.

    If otherwise:
    ```bash
    sudo find /var/www/html/wordpress/ -type d -exec chmod 755 {} \;
    sudo find /var/www/html/wordpress/ -type f -exec chmod 644 {} \;
    ```
4.  **Use rsync**:
    ```
    rsync -avz --delete /var/www/html/wordpress/ /backups/wordpress/
    ```
5.  **Automate**:
    ```bash
    crontab -e
    ```
    then in vim:
    press i to insert/edit
    ```vim
    59 23 * * * rsync -avz --delete /var/www/html/wordpress/ /backups/wordpress/
    ```
    then press `esc` to exit insert mode
    then type `:wq` and press `enter` to save and exit
6.  **Create compressed file**:
    ```
    tar -zcvf /backups/wordpress.tar.gz /backups/wordpress/
    ```

### **Create a compressed archive of the /backups/wordpress/ directory**
    It should be possible to restore the directory with this command: sudo tar xpvf /backups/wordpress.tar.* 
    ```
    sudo tar cvpf /backups/wordpress.tar.gz wordpress
    ```

 Configure a firewall to:
a. Allow all network access for ssh on port 22
b. Reject all remote machines network access for mysql on its service port
c. Allow all network access for your unpopulated WordPress website

Drop incoming connections from all unused ports for the default zone

Ensure that Apache httpd has read and write access to all files within

Configure SELinux security contexts for /srv/wordpress/ to persist after a reboot

Ensure that WordPress can update itself with SELinux enabled, even after a reboot

**Deploy Drupal**

Deploy Drupal to the /var/www/html/drupal/ directory

Install Drupal and configure it with the Standard installation profile

Ensure that Drupal is fully working after completion


# Cyber Enhancement Module


##**Encryption and Decryption**

**Encryption:**

- Encryption is the process of converting plaintext into ciphertext.
- It uses an algorithm and a key to transform the data.
- The purpose is to protect the data from unauthorized access.
- In the context of the Caesar cipher, encryption involves shifting the letters of the plaintext by a certain number of positions in the alphabet.

**Decryption:**

- Decryption is the process of converting ciphertext back into plaintext.
- It uses the same or a related algorithm and key used for encryption.
- The purpose is to retrieve the original data for authorized use.
- In the Caesar cipher, decryption involves shifting the letters of the ciphertext back by the same number of positions used in the encryption.

**Caesar Cipher:**

- The Caesar cipher is a type of substitution cipher where each letter in the plaintext is shifted a certain number of places down or up the alphabet.
- For example, with a shift of 1, 'A' would be encrypted as 'B', 'B' as 'C', and so on.
- Decryption is done by shifting in the opposite direction.
- It's one of the simplest forms of encryption and can be easily broken with frequency analysis or by brute force.

##**Encryption Keys**

- Encryption keys are a fundamental part of cryptography, used to encrypt and decrypt data.

- They are a series of numbers and/or letters, typically generated by an algorithm.

- There are two main types of encryption keys: symmetric and asymmetric.

  - Symmetric keys: Same key is used for both encryption and decryption.
    - Example: Advanced Encryption Standard (AES) uses symmetric keys.
    - Use Case: Encrypting data at rest, such as files on a hard drive.

  - Asymmetric keys: Two different keys are used - a public key for encryption and a private key for decryption.
    - Example: RSA (Rivest–Shamir–Adleman) uses asymmetric keys.
    - Use Case: Secure communication over the internet, such as HTTPS.

- Key length is an important factor in security. The longer the key, the harder it is for someone to break the encryption.

- Key management is a critical aspect of encryption. This involves the secure generation, storage, exchange and use of keys.

- Keys should be replaced periodically, a practice known as key rotation, to enhance security.

- Encryption keys are used in various applications, from securing internet communications, to protecting sensitive data in databases, to encrypting files on personal devices.

- Mismanagement or loss of keys can lead to data loss or breaches, highlighting the importance of secure key management practices.

##**The Weakest Factor**
The weakest factor in cybersecurity is often considered to be the human element. Here's a description in bullet points:

- Human Error: People can unintentionally cause security breaches by falling for phishing scams, using weak passwords, or misconfiguring security settings.

- Social Engineering: This is a technique used by attackers to manipulate individuals into giving up sensitive information. It's successful because it exploits human psychology and trust.
  - Example: A common type of social engineering is a phishing email, where the attacker pretends to be a trusted entity (like a bank or a friend) and convinces the victim to reveal sensitive data or click on a malicious link.

- Lack of Training: Employees who are not trained in cybersecurity best practices are more likely to make mistakes that lead to breaches.

- Insider Threats: Sometimes, the threat comes from within the organization. A disgruntled employee, for example, might intentionally harm the company's systems or leak sensitive data.

- Password Misuse: Using weak passwords, sharing passwords, or not changing default passwords are common issues.
  - Example: Passwords like "123456" or "password" are incredibly easy for attackers to guess.

- Poor Patch Management: Not updating or patching software and systems in a timely manner can leave known vulnerabilities open for exploitation.

- Techniques to Mitigate Risk:
  - Regular Training: Continually educate employees about the latest threats and how to avoid them.
  - Use Strong, Unique Passwords: Implement a robust password policy in the organization.
  - Regular Patching and Updates: Ensure all systems and software are up-to-date.
  - Implement Multi-Factor Authentication: This adds an extra layer of security, making it harder for attackers to gain access even if they have the password.
  - Regular Audits and Monitoring: Keep a keen eye on system activities to detect any suspicious behavior early.
